### Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C3700        1
C1728        1
C2380        1
C2570        1
C1580        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
income_count = application_df.INCOME_AMT.value_counts()
income_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_income = list(income_count[income_count < 300].index)

# Replace in dataframe
for cls in replace_income:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [10]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                7
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop('IS_SUCCESSFUL',1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3360      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,821
Trainable params: 5,821
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5880 - accuracy: 0.7102
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7335
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5511 - accuracy: 0.7339
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5530 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7317
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5504 - accuracy: 0.7324
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5424 - accuracy: 0.7382
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7326
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5452 - accuracy: 0.7358
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accura

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5619 - accuracy: 0.7244
Loss: 0.5619034767150879, Accuracy: 0.7244315147399902


In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization1/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization1/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 90/804 [==>...........................] - ETA: 0s - loss: 0.5335 - accuracy: 0.7411
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
167/804 [=====>........................] - ETA: 0s - loss: 0.5308 - accuracy: 0.7430
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
289/804 [=========>....................] - ETA: 0s - loss: 0.5317 - accuracy: 0.7429
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
394/804 [=============>................] - ETA: 0s - loss: 0.5325 - accuracy: 0.7423
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
463/804 [================>.............] - ETA: 0s - loss: 0.5330 - accuracy: 0.7420
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
583/804 [====================>.........] - ETA: 0s - loss: 0.5336 - accuracy: 0.7416
Epoch 00001: saving model to checkpoints/Optimization1/weights.01.hdf5
680/804 [========================>.....] - ETA: 0s -

110/804 [===>..........................] - ETA: 0s - loss: 0.5240 - accuracy: 0.7441
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
244/804 [========>.....................] - ETA: 0s - loss: 0.5273 - accuracy: 0.7426
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
326/804 [===========>..................] - ETA: 0s - loss: 0.5272 - accuracy: 0.7433
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
448/804 [===============>..............] - ETA: 0s - loss: 0.5279 - accuracy: 0.7431
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
527/804 [==================>...........] - ETA: 0s - loss: 0.5284 - accuracy: 0.7429
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
608/804 [=====================>........] - ETA: 0s - loss: 0.5286 - accuracy: 0.7429
Epoch 00013: saving model to checkpoints/Optimization1/weights.13.hdf5
742/804 [==========================>...] - ETA: 0s - loss: 0.529

194/804 [======>.......................] - ETA: 0s - loss: 0.5345 - accuracy: 0.7382
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
319/804 [==========>...................] - ETA: 0s - loss: 0.5336 - accuracy: 0.7398
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
395/804 [=============>................] - ETA: 0s - loss: 0.5334 - accuracy: 0.7403
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
518/804 [==================>...........] - ETA: 0s - loss: 0.5328 - accuracy: 0.7408
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
589/804 [====================>.........] - ETA: 0s - loss: 0.5327 - accuracy: 0.7410
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
717/804 [=========================>....] - ETA: 0s - loss: 0.5325 - accuracy: 0.7414
Epoch 00019: saving model to checkpoints/Optimization1/weights.19.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0

301/804 [==========>...................] - ETA: 0s - loss: 0.5317 - accuracy: 0.7385
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
383/804 [=============>................] - ETA: 0s - loss: 0.5315 - accuracy: 0.7387
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
464/804 [================>.............] - ETA: 0s - loss: 0.5311 - accuracy: 0.7390
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
599/804 [=====================>........] - ETA: 0s - loss: 0.5312 - accuracy: 0.7394
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
674/804 [========================>.....] - ETA: 0s - loss: 0.5315 - accuracy: 0.7395
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
801/804 [============================>.] - ETA: 0s - loss: 0.5319 - accuracy: 0.7397
Epoch 00025: saving model to checkpoints/Optimization1/weights.25.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0

363/804 [============>.................] - ETA: 0s - loss: 0.5386 - accuracy: 0.7367
Epoch 00031: saving model to checkpoints/Optimization1/weights.31.hdf5
480/804 [================>.............] - ETA: 0s - loss: 0.5380 - accuracy: 0.7375
Epoch 00031: saving model to checkpoints/Optimization1/weights.31.hdf5
554/804 [===================>..........] - ETA: 0s - loss: 0.5377 - accuracy: 0.7379
Epoch 00031: saving model to checkpoints/Optimization1/weights.31.hdf5
666/804 [=======================>......] - ETA: 0s - loss: 0.5373 - accuracy: 0.7385
Epoch 00031: saving model to checkpoints/Optimization1/weights.31.hdf5
779/804 [============================>.] - ETA: 0s - loss: 0.5369 - accuracy: 0.7388
Epoch 00031: saving model to checkpoints/Optimization1/weights.31.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7388
Epoch 32/100
 40/804 [>.............................] - ETA: 0s - loss: 0.5302 - accuracy: 0.7351
Epoch 00032: saving model to chec

...

556/804 [===================>..........] - ETA: 0s - loss: 0.5311 - accuracy: 0.7430
Epoch 00055: saving model to checkpoints/Optimization1/weights.55.hdf5
675/804 [========================>.....] - ETA: 0s - loss: 0.5316 - accuracy: 0.7424
Epoch 00055: saving model to checkpoints/Optimization1/weights.55.hdf5
758/804 [===========================>..] - ETA: 0s - loss: 0.5317 - accuracy: 0.7422
Epoch 00055: saving model to checkpoints/Optimization1/weights.55.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5318 - accuracy: 0.7421
Epoch 56/100
 48/804 [>.............................] - ETA: 0s - loss: 0.5339 - accuracy: 0.7484
Epoch 00056: saving model to checkpoints/Optimization1/weights.56.hdf5
172/804 [=====>........................] - ETA: 0s - loss: 0.5321 - accuracy: 0.7442
Epoch 00056: saving model to checkpoints/Optimization1/weights.56.hdf5
256/804 [========>.....................] - ETA: 0s - loss: 0.5317 - accuracy: 0.7434
Epoch 00056: saving model to chec

624/804 [======================>.......] - ETA: 0s - loss: 0.5301 - accuracy: 0.7402
Epoch 00067: saving model to checkpoints/Optimization1/weights.67.hdf5
728/804 [==========================>...] - ETA: 0s - loss: 0.5306 - accuracy: 0.7402
Epoch 00067: saving model to checkpoints/Optimization1/weights.67.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5308 - accuracy: 0.7404
Epoch 68/100
 27/804 [>.............................] - ETA: 1s - loss: 0.5316 - accuracy: 0.7416
Epoch 00068: saving model to checkpoints/Optimization1/weights.68.hdf5
 95/804 [==>...........................] - ETA: 1s - loss: 0.5371 - accuracy: 0.7419
Epoch 00068: saving model to checkpoints/Optimization1/weights.68.hdf5
212/804 [======>.......................] - ETA: 1s - loss: 0.5323 - accuracy: 0.7452
Epoch 00068: saving model to checkpoints/Optimization1/weights.68.hdf5
317/804 [==========>...................] - ETA: 0s - loss: 0.5313 - accuracy: 0.7451
Epoch 00068: saving model to chec

709/804 [=========================>....] - ETA: 0s - loss: 0.5348 - accuracy: 0.7382
Epoch 00073: saving model to checkpoints/Optimization1/weights.73.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5344 - accuracy: 0.7386
Epoch 74/100
  1/804 [..............................] - ETA: 1s - loss: 0.5907 - accuracy: 0.6875
Epoch 00074: saving model to checkpoints/Optimization1/weights.74.hdf5
 81/804 [==>...........................] - ETA: 1s - loss: 0.5526 - accuracy: 0.7279
Epoch 00074: saving model to checkpoints/Optimization1/weights.74.hdf5
197/804 [======>.......................] - ETA: 1s - loss: 0.5411 - accuracy: 0.7408
Epoch 00074: saving model to checkpoints/Optimization1/weights.74.hdf5
305/804 [==========>...................] - ETA: 0s - loss: 0.5378 - accuracy: 0.7440
Epoch 00074: saving model to checkpoints/Optimization1/weights.74.hdf5
387/804 [=============>................] - ETA: 0s - loss: 0.5366 - accuracy: 0.7452
Epoch 00074: saving model to chec

661/804 [=======================>......] - ETA: 0s - loss: 0.5325 - accuracy: 0.7413
Epoch 00085: saving model to checkpoints/Optimization1/weights.85.hdf5
746/804 [==========================>...] - ETA: 0s - loss: 0.5324 - accuracy: 0.7414
Epoch 00085: saving model to checkpoints/Optimization1/weights.85.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5325 - accuracy: 0.7414
Epoch 86/100
 60/804 [=>............................] - ETA: 1s - loss: 0.5376 - accuracy: 0.7390
Epoch 00086: saving model to checkpoints/Optimization1/weights.86.hdf5
144/804 [====>.........................] - ETA: 1s - loss: 0.5341 - accuracy: 0.7408
Epoch 00086: saving model to checkpoints/Optimization1/weights.86.hdf5
236/804 [=======>......................] - ETA: 0s - loss: 0.5322 - accuracy: 0.7418
Epoch 00086: saving model to checkpoints/Optimization1/weights.86.hdf5
329/804 [===========>..................] - ETA: 0s - loss: 0.5315 - accuracy: 0.7426
Epoch 00086: saving model to chec

697/804 [=========================>....] - ETA: 0s - loss: 0.5279 - accuracy: 0.7446
Epoch 00097: saving model to checkpoints/Optimization1/weights.97.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5284 - accuracy: 0.7441
Epoch 98/100
  1/804 [..............................] - ETA: 2s - loss: 0.6607 - accuracy: 0.6250
Epoch 00098: saving model to checkpoints/Optimization1/weights.98.hdf5
 76/804 [=>............................] - ETA: 1s - loss: 0.5540 - accuracy: 0.7274
Epoch 00098: saving model to checkpoints/Optimization1/weights.98.hdf5
183/804 [=====>........................] - ETA: 1s - loss: 0.5462 - accuracy: 0.7308
Epoch 00098: saving model to checkpoints/Optimization1/weights.98.hdf5
287/804 [=========>....................] - ETA: 0s - loss: 0.5430 - accuracy: 0.7329
Epoch 00098: saving model to checkpoints/Optimization1/weights.98.hdf5
411/804 [==============>...............] - ETA: 0s - loss: 0.5408 - accuracy: 0.7349
Epoch 00098: saving model to chec

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5686 - accuracy: 0.7242
Loss: 0.5685877203941345, Accuracy: 0.7241982221603394


In [21]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization1.h5')